# 第 5 章 交易环境

在这一章中，我们将研究怎样搭建量化交易的强化学习环境。

## 读入数据
我们首先从CSV文件中读入数据：
```python
data_path = pathlib.Path(AppConfig.STOCKS)
val_path = pathlib.Path(AppConfig.VAL_STOCKS)
year = 2016

if year is not None or data_path.is_file():
    if year is not None:
        print('load stock data...')
        stock_data = BarData.load_year_data(year)
    else:
        stock_data = {"YNDX": BarData.load_relative(data_path)}
    print('stodk_data: {0};'.format(stock_data))
    env = HourBarEnv(
        stock_data, bars_count=AppConfig.BARS_COUNT)
    env_tst = HourBarEnv(
        stock_data, bars_count=AppConfig.BARS_COUNT)
elif data_path.is_dir():
    env = HourBarEnv.from_dir(
        data_path, bars_count=AppConfig.BARS_COUNT)
    env_tst = HourBarEnv.from_dir(
        data_path, bars_count=AppConfig.BARS_COUNT)
else:
    raise RuntimeError("No data to train on")
```
运行结果为：
```
stodk_data: {'data\\YNDX_160101_161231.csv': BarPrices(open=array([1156.9, 1150.6, 1150.2, ..., 1245.5, 1246. , 1244. ], dtype=float32), high=array([0.00086438, 0.        , 0.        , ..., 0.        , 0.        ,
       0.00361736], dtype=float32), low=array([-0.0033711 , -0.00017378, -0.00060855, ..., -0.00080289,
       -0.00160514, -0.00040193], dtype=float32), close=array([-0.0033711 , -0.00017378, -0.00043471, ..., -0.00080289,
       -0.00080257,  0.00361736], dtype=float32), volume=array([ 43.,   5., 165., ..., 200., 231., 191.], dtype=float32))};
```
如上所示，stock_data为一个字典，BarPrices是Python3的collections.namedtuple，属性为：open,、high,、low,、close,、volume，其值均为np.ndarray。其处理方法为：对于日内数据，将最高、最低、收盘均减去开盘价再除以开盘价，用$o$代表开盘价、$h$代表最高价、$l$代表最低价、$c$代表最低价，计算公式为：
$$
\hat{h}=\frac{h-o}{o}, \hat{l}=\frac{l-o}{o}, \hat{c}=\frac{c-o}{o}
$$
采用这种处理方式，虽然可以更好的反映价格变动，但是要计算金额等量时，就会非常不方便，因此我们还需要返回当前时刻的原始值，因此我们将改造biz.drlt.envs.minute_bar_env.State.encode方法，在返回的obs中加入这些数据。

## 分钟线市场环境
接下来，我们根据stock_data数据，生成分钟线强化学习环境。
我们先来看环境状态State类，其是环境的内部状态，通过encode方法，可以返回Agent可以观察到的Observation。这个类是环境类的核心，所以我们需要重点来理解。我们先来构造函数，环境初始化重置，获取初始时的obs：

In [ ]:
# c001
class State:
    def __init__(self, bars_count, commission_perc,
                 reset_on_close, reward_on_close=True,
                 volumes=True):
        assert isinstance(bars_count, int)
        assert bars_count > 0
        assert isinstance(commission_perc, float)
        assert commission_perc >= 0.0
        assert isinstance(reset_on_close, bool)
        assert isinstance(reward_on_close, bool)
        self.bars_count = bars_count
        self.commission_perc = commission_perc
        self.reset_on_close = reset_on_close
        self.reward_on_close = reward_on_close
        self.volumes = volumes

    def reset(self, prices, offset):
        assert isinstance(prices, BarData.BarPrices)
        assert offset >= self.bars_count-1
        self.have_position = False
        self.open_price = 0.0
        self._prices = prices
        self._offset = offset

    @property
    def shape(self):
        # [h, l, c] * bars + position_flag + rel_profit
        if self.volumes:
            return 4 * self.bars_count + 1 + 1,
        else:
            return 3*self.bars_count + 1 + 1,

    def encode(self):
        """
        Convert current state into numpy array.
        """
        res = np.ndarray(shape=self.shape, dtype=np.float32)
        shift = 0
        for bar_idx in range(-self.bars_count+1, 1):
            ofs = self._offset + bar_idx
            res[shift] = self._prices.high[ofs]
            shift += 1
            res[shift] = self._prices.low[ofs]
            shift += 1
            res[shift] = self._prices.close[ofs]
            shift += 1
            if self.volumes:
                res[shift] = self._prices.volume[ofs]
                shift += 1
        res[shift] = float(self.have_position)
        shift += 1
        if not self.have_position:
            res[shift] = 0.0
        else:
            res[shift] = self._cur_close() / self.open_price - 1.0
        return res

    def test_State_main(self):
        print('生成环境状态类')
        year = 2016
        instrument = 'data\\YNDX_160101_161231.csv'
        stock_data = BarData.load_year_data(year)
        print('stock_data: {0};'.format(stock_data[instrument]))
        st = State(bars_count=10, commission_perc=0.1, reset_on_close=True, reward_on_close=True,volumes=True)
        st.reset(stock_data[instrument], offset=AppConfig.BARS_COUNT+1)
        obs = st.encode()
        print('initial observation: type:{0}; shape:{1};'.format(type(obs), obs))
        action = AssetActions.Buy
        reward, done = st.step(action=action)
        print('reward={0}; done={1};'.format(reward, done))
        obs = st.encode()
        info = {
            'instrument': 'YNDX',
            'offset': st._offset
        }
        print('********************** step **********************************')
        print('obs: {0};'.format(obs))
        print('info: {0};'.format(info))
        #while True:
        #    st.step()
        print('^_^')
        self.assertTrue(1>0)

代码解读如下所示：
* 第20行：在c001中，定义当前为空仓，即未持有股票；
* 第22行：应该是购买股票时的价格；
* 第24行：我们在第66行重置环境时，self._offset的值为AppConfig.BARS_COUNT+1=11；
* 第38行：res为42个元素的一维数组，前40行内容为：最高价、最低价、收盘价、交易量，其中价格是减去开盘价再除以开盘价，第41位表示是否有持仓，第42位是收益率，当前收盘价也开仓价再减1代表收益率；
* 第41行：elf._offset=11，self.bar_count=10，所以第一次时bar_idx=-9，因此ofs=2；
* 第42$\sim$47行：将计算后的最高价、最低价、收盘价添加到res数组中；
* 第48$\sim$50行：如果具有交易量，则将交易量加入到res数组中；
* 第51、52行：标志是否持有股票；
* 第53$\sim$56行：计算收益率，加入到res数组中；

下面我们来看购买和卖出的业务逻辑：

In [ ]:
class State:

    def _cur_close(self):
        """
        Calculate real close price for the current bar
        """
        open = self._prices.open[self._offset]
        rel_close = self._prices.close[self._offset]
        return open * (1.0 + rel_close)

    def step(self, action):
        """
        Perform one step in our price, adjust offset, check for the end of prices
        and handle position change
        :param action:
        :return: reward, done
        """
        assert isinstance(action, AssetActions)
        reward = 0.0
        done = False
        close = self._cur_close()
        if action == AssetActions.Buy and not self.have_position:
            self.have_position = True
            self.open_price = close
            reward -= self.commission_perc
        elif action == AssetActions.Close and self.have_position:
            reward -= self.commission_perc
            done |= self.reset_on_close
            if self.reward_on_close:
                reward += 100.0 * (close / self.open_price - 1.0)
            self.have_position = False
            self.open_price = 0.0
        self._offset += 1
        prev_close = close
        close = self._cur_close()
        done |= self._offset >= self._prices.close.shape[0]-1
        if self.have_position and not self.reward_on_close:
            reward += 100.0 * (close / prev_close - 1.0)
        return reward, done
    
    def test_State_main(self):
        print('生成环境状态类')
        year = 2016
        instrument = 'data\\YNDX_160101_161231.csv'
        stock_data = BarData.load_year_data(year)
        print('stock_data: {0};'.format(stock_data[instrument]))
        st = State(bars_count=10, commission_perc=0.1, reset_on_close=True, reward_on_close=True,volumes=True)
        st.reset(stock_data[instrument], offset=AppConfig.BARS_COUNT+1)
        obs = st.encode()
        print('initial observation: type:{0}; shape:{1};'.format(type(obs), obs))
        action = AssetActions.Buy
        reward, done = st.step(action=action)
        print('reward={0}; done={1};'.format(reward, done))
        obs = st.encode()
        info = {
            'instrument': 'YNDX',
            'offset': st._offset
        }
        print('********************** step **********************************')
        print('obs: {0};'.format(obs))
        print('info: {0};'.format(info))
        #while True:
        #    st.step()
        print('^_^')
        self.assertTrue(1>0)    